<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Goal" data-toc-modified-id="Goal-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Goal</a></span></li><li><span><a href="#Var" data-toc-modified-id="Var-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Var</a></span></li><li><span><a href="#Init" data-toc-modified-id="Init-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Init</a></span><ul class="toc-item"><li><span><a href="#Functions" data-toc-modified-id="Functions-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Functions</a></span></li></ul></li><li><span><a href="#Load" data-toc-modified-id="Load-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Load</a></span></li><li><span><a href="#Count-table" data-toc-modified-id="Count-table-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Count table</a></span><ul class="toc-item"><li><span><a href="#Metadata" data-toc-modified-id="Metadata-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Metadata</a></span></li><li><span><a href="#Beta-diversity" data-toc-modified-id="Beta-diversity-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Beta diversity</a></span></li></ul></li><li><span><a href="#dbRDA" data-toc-modified-id="dbRDA-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>dbRDA</a></span><ul class="toc-item"><li><span><a href="#Westernization-&amp;-gender" data-toc-modified-id="Westernization-&amp;-gender-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Westernization &amp; gender</a></span><ul class="toc-item"><li><span><a href="#Bray-curtis" data-toc-modified-id="Bray-curtis-6.1.1"><span class="toc-item-num">6.1.1&nbsp;&nbsp;</span>Bray-curtis</a></span></li><li><span><a href="#wunifrac-phy" data-toc-modified-id="wunifrac-phy-6.1.2"><span class="toc-item-num">6.1.2&nbsp;&nbsp;</span>wunifrac-phy</a></span></li><li><span><a href="#wunifrac-trt" data-toc-modified-id="wunifrac-trt-6.1.3"><span class="toc-item-num">6.1.3&nbsp;&nbsp;</span>wunifrac-trt</a></span></li></ul></li><li><span><a href="#Westernization-&amp;-gender-+-age-+-BMI" data-toc-modified-id="Westernization-&amp;-gender-+-age-+-BMI-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Westernization &amp; gender + age + BMI</a></span><ul class="toc-item"><li><span><a href="#Bray-Curtis" data-toc-modified-id="Bray-Curtis-6.2.1"><span class="toc-item-num">6.2.1&nbsp;&nbsp;</span>Bray-Curtis</a></span></li><li><span><a href="#wunifrac-phy" data-toc-modified-id="wunifrac-phy-6.2.2"><span class="toc-item-num">6.2.2&nbsp;&nbsp;</span>wunifrac-phy</a></span></li><li><span><a href="#wunifrac-trt" data-toc-modified-id="wunifrac-trt-6.2.3"><span class="toc-item-num">6.2.3&nbsp;&nbsp;</span>wunifrac-trt</a></span></li></ul></li></ul></li><li><span><a href="#sessionInfo" data-toc-modified-id="sessionInfo-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>sessionInfo</a></span></li></ul></div>

# Goal

* distance-based RDA on phylogenies

# Var

In [1]:
# working dir
work_dir = '/ebio/abt3_projects/databases_no-backup/curatedMetagenomicData/global_metagenomes/diversity/'

# bracken counts
brk_file = '/ebio/abt3_projects/databases_no-backup/curatedMetagenomicData/global_metagenomes/diversity/bracken_filt.qs'

# metadata
## filtered
metadata_filt_file = file.path(work_dir, 'CurMetDat_metadata_filt-n1846.tsv')
## all 
base_in_dir = '/ebio/abt3_projects/small_projects/nyoungblut/public_data_retireval/'
metadata_file = file.path(base_in_dir, 'CurMetDat-metagenomes', 'files', 'metadata', 'Filtered_CurMetDat_f.tsv')

# ref genome metadata (GTDBr89 metadata)
genome_meta_file = '/ebio/abt3_projects/databases_no-backup/GTDB/release89/LLMGP-DB/metadata_1per-GTDB-Spec_gte50comp-lt5cont_wtaxID_wPath.tsv'

# trees
## GTDB phylogeny
phy_file = '/ebio/abt3_projects/databases_no-backup/GTDB/release89/LLMGP-DB/phylogeny/ar122-bac120_r89_1per-GTDB-Spec_gte50comp-lt5cont_rn.nwk'
## trait phylogeny
phy_trt_file = '/ebio/abt3_projects/databases_no-backup/GTDB/release89/LLMGP-DB/genomes/LLG//phenotype/predictions_flat_majority-votes_combined_jaccard_rn.nwk'

# beta-diversity
dist_names1 = list('bray'='bray', 'jaccard'='jaccard', 
                  'wunifrac'='wunifrac', 'unifrac'='unifrac')
dist_names2 = list('wunifrac'='wunifrac', 'unifrac'='unifrac')
dist_F1 = '/ebio/abt3_projects/databases_no-backup/curatedMetagenomicData/global_metagenomes/diversity/beta-div.qs'
dist_F2 = '/ebio/abt3_projects/databases_no-backup/curatedMetagenomicData/global_metagenomes/diversity/beta-div_trt-maj.qs'

# params
threads = 16

# Init

In [4]:
library(dplyr)
library(tidyr)
library(ggplot2)
library(data.table)
library(tidytable)
library(ape)
library(phyloseq)
library(vegan)
library(LeyLabRMisc)

In [5]:
df.dims()
setDTthreads(threads)
RhpcBLASctl::blas_set_num_threads(1)
make_dir(work_dir)

Directory already exists: /ebio/abt3_projects/databases_no-backup/curatedMetagenomicData/global_metagenomes/diversity/ 


## Functions

In [20]:
filter_dist = function(d, x){
    d = as.matrix(d)
    x = intersect(rownames(d), x)
    d = as.dist(d[x,x])
    return(d)
}

# Load

# Count table

In [6]:
# loading as "wide" table
brk_cls = qs::qread(brk_file, nthreads=threads) %>%
    dt_mutate(name = gsub(' ', '_', name)) %>%
    dt_select(name, Sample, Abundance) %>%
    dt_pivot_wider(names_from=Sample, values_from=Abundance) %>%
    dt_mutate_across(is.numeric, function(x) ifelse(is.na(x), 0, x))
brk_cls

Warning message:
“Predicate functions must be wrapped in `where()`.

  # Bad
  data %>% select(is.numeric)

  # Good
  data %>% select(where(is.numeric))

ℹ Please update your code.
This message is displayed once per session.”


name,A01_02_1FE,A02_01_1FE,A03_01_1FE,A04_01_1FE,A04_04_1FE,A05_01_1FE,A06_01_1FE,A07_01_1FE,A08_01_1FE,⋯,nHM512828,nHM512834,nHM611708,nHM612607,nHM612709,nHM612724,nHM612743,nHM612836,nHMX11726,scavenger.11.60.0
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
s__1-14-0-10-34-76_sp002763075,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
s__1-19_sp001922585,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
s__Zoogloea_sp002028455,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0.00014,0,0
s__Zunongwangia_mangrovi,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0.00000,0,0


In [7]:
# formatting 
x = brk_cls$name
brk_cls$name = NULL
brk_cls = brk_cls %>% t %>% as.data.frame
colnames(brk_cls) = x
brk_cls

,s__1-14-0-10-34-76_sp002763075,s__1-19_sp001922585,s__14-2_sp000403255,s__14-2_sp000403315,s__14-2_sp000403845,s__14-2_sp001940225,s__2-01-FULL-39-21_sp001792305,s__2013-40CM-41-45_sp002215675,s__46-32_sp003243445,s__51-20_sp001917175,⋯,s__Zag111_sp002405805,s__Zag111_sp002438405,s__Zag111_sp003258735,s__Zag1_sp000433095,s__Zag1_sp000437435,s__Zag1_sp000438175,s__Zag1_sp001765415,s__Zag1_sp001917115,s__Zoogloea_sp002028455,s__Zunongwangia_mangrovi
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
A01_02_1FE,0,0,0,0,0,0,0,0,0,0,⋯,0,0.00000,0,0,0,0,0.00000,0,0,0
A02_01_1FE,0,0,0,0,0,0,0,0,0,0,⋯,0,0.00036,0,0,0,0,0.00038,0,0,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
nHMX11726,0,0,0,0,0,0e+00,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
scavenger.11.60.0,0,0,0,0,0,3e-04,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


## Metadata

In [8]:
# all 
meta = Fread(metadata_file) %>%
    dt_mutate(Sample = gsub('-', '.', Sample)) %>%
    dt_filter(number_reads >= 5e5) %>%
    dt_mutate(dataset_name = dataset_name %>% as.factor,
              study_condition = study_condition %>% as.factor)
meta %>% unique_n('Samples', Sample)
meta

No. of unique Samples: 3582 


dataset_name,Sample,subjectID,body_site,antibiotics_current_use,study_condition,disease,age,age_category,gender,⋯,il_1,glp_1,travel_destination,urea_nitrogen,disease_stage,uncurated_metadata,family_member,subject_n,samp_n,westernized
<fct>,<chr>,<chr>,<chr>,<chr>,<fct>,<chr>,<int>,<chr>,<chr>,⋯,<dbl>,<dbl>,<chr>,<dbl>,<int>,<chr>,<chr>,<int>,<int>,<lgl>
AsnicarF_2017,MV_FEM1_t1Q14,MV_FEM1,stool,NA,control,healthy,NA,adult,female,⋯,NA,NA,NA,NA,NA,NA,NA,1,1,TRUE
AsnicarF_2017,MV_FEM2_t1Q14,MV_FEM2,stool,NA,control,healthy,NA,adult,female,⋯,NA,NA,NA,NA,NA,NA,NA,1,1,TRUE
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
ZellerG_2014,CCIS82944710ST.20.0,FR-730,stool,NA,control,healthy,38,adult,female,⋯,NA,NA,NA,NA,NA,NA,NA,1,1,TRUE
ZellerG_2014,CCIS93040568ST.20.0,FR-682,stool,NA,control,healthy,65,adult,male,⋯,NA,NA,NA,NA,NA,NA,NA,1,1,TRUE


## Beta diversity

In [9]:
# loading as a list of distance matrices
D = list('bray' = qs::qread(dist_F1, nthreads=4)[['bray']],
         'jaccard' = qs::qread(dist_F1, nthreads=4)[['jaccard']],
         'unifrac_phy' = qs::qread(dist_F1, nthreads=4)[['unifrac']],
         'wunifrac_phy' = qs::qread(dist_F1, nthreads=4)[['wunifrac']],
         'unifrac_trt' = qs::qread(dist_F2, nthreads=4)[['unifrac']],
         'wunifrac_trt' = qs::qread(dist_F2, nthreads=4)[['wunifrac']])
D %>% names

[1] "bray"         "jaccard"      "unifrac_phy"  "wunifrac_phy" "unifrac_trt" 
[6] "wunifrac_trt"

In [10]:
# number of samples per dist
D %>%
    lapply(function(x) x %>% labels %>% length)

$bray
[1] 3348

$jaccard
[1] 3348

$unifrac_phy
[1] 3348

$wunifrac_phy
[1] 3348

$unifrac_trt
[1] 3348

$wunifrac_trt
[1] 3348

# dbRDA

## Westernization & gender

In [59]:
# formatting metadata
m = meta %>%
    dt_filter(!is.na(westernized), !is.na(gender)) %>%
    as.data.frame
rownames(m) = m$Sample
m

,dataset_name,Sample,subjectID,body_site,antibiotics_current_use,study_condition,disease,age,age_category,gender,⋯,il_1,glp_1,travel_destination,urea_nitrogen,disease_stage,uncurated_metadata,family_member,subject_n,samp_n,westernized
,<fct>,<chr>,<chr>,<chr>,<chr>,<fct>,<chr>,<int>,<chr>,<chr>,⋯,<dbl>,<dbl>,<chr>,<dbl>,<int>,<chr>,<chr>,<int>,<int>,<lgl>
MV_FEM1_t1Q14,AsnicarF_2017,MV_FEM1_t1Q14,MV_FEM1,stool,NA,control,healthy,NA,adult,female,⋯,NA,NA,NA,NA,NA,NA,NA,1,1,TRUE
MV_FEM2_t1Q14,AsnicarF_2017,MV_FEM2_t1Q14,MV_FEM2,stool,NA,control,healthy,NA,adult,female,⋯,NA,NA,NA,NA,NA,NA,NA,1,1,TRUE
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
CCIS82944710ST.20.0,ZellerG_2014,CCIS82944710ST.20.0,FR-730,stool,NA,control,healthy,38,adult,female,⋯,NA,NA,NA,NA,NA,NA,NA,1,1,TRUE
CCIS93040568ST.20.0,ZellerG_2014,CCIS93040568ST.20.0,FR-682,stool,NA,control,healthy,65,adult,male,⋯,NA,NA,NA,NA,NA,NA,NA,1,1,TRUE


### Bray-curtis

In [61]:
# filtering & ordering
## distance matrix
d = filter_dist(D[['bray']], rownames(m))
d %>% labels %>% length
## metadata
M = m[labels(d),]
M

[1] 2154

,dataset_name,Sample,subjectID,body_site,antibiotics_current_use,study_condition,disease,age,age_category,gender,⋯,il_1,glp_1,travel_destination,urea_nitrogen,disease_stage,uncurated_metadata,family_member,subject_n,samp_n,westernized
,<fct>,<chr>,<chr>,<chr>,<chr>,<fct>,<chr>,<int>,<chr>,<chr>,⋯,<dbl>,<dbl>,<chr>,<dbl>,<int>,<chr>,<chr>,<int>,<int>,<lgl>
A11_01_1FE,PasolliE_2018,A11_01_1FE,A11_01_1FE,stool,no,control,healthy,52,adult,male,⋯,NA,NA,NA,NA,NA,NA,NA,1,1,FALSE
A13_01_1FE,PasolliE_2018,A13_01_1FE,A13_01_1FE,stool,no,control,healthy,52,adult,male,⋯,NA,NA,NA,NA,NA,NA,NA,1,1,FALSE
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
YSZC12003_37879,XieH_2016,YSZC12003_37879,YSZC12003_37879,stool,NA,control,NA,42,adult,female,⋯,NA,NA,NA,NA,NA,NA,NA,1,1,TRUE
YSZC12003_37880,XieH_2016,YSZC12003_37880,YSZC12003_37880,stool,NA,control,NA,42,adult,female,⋯,NA,NA,NA,NA,NA,NA,NA,1,1,TRUE


In [62]:
# distance-based RDA
dbRDA = capscale(d ~ westernized + gender, data = M)
dbRDA

Call: capscale(formula = d ~ westernized + gender, data = M)

                 Inertia Proportion Rank
Total          655.35110    1.00000     
Constrained     48.83299    0.07451    2
Unconstrained  784.37189    1.19687  808
Imaginary     -177.85378   -0.27139 1345
Inertia is squared Unknown distance 

Eigenvalues for constrained axes:
 CAP1  CAP2 
45.73  3.11 

Eigenvalues for unconstrained axes:
 MDS1  MDS2  MDS3  MDS4  MDS5  MDS6  MDS7  MDS8 
74.79 60.76 32.10 25.04 17.63 16.70 12.95 12.38 
(Showing 8 of 808 unconstrained eigenvalues)


In [63]:
# anova
anova.cca(dbRDA, by='terms')

,Df,SumOfSqs,F,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>
westernized,1,45.241599,124.067015,0.001
gender,1,3.591387,9.848737,0.001
Residual,2151,784.371892,NA,NA


### wunifrac-phy

In [64]:
# filtering & ordering
## distance matrix
d = filter_dist(D[['wunifrac_phy']], rownames(m))
d %>% labels %>% length
## metadata
M = m[labels(d),]
M

[1] 2154

,dataset_name,Sample,subjectID,body_site,antibiotics_current_use,study_condition,disease,age,age_category,gender,⋯,il_1,glp_1,travel_destination,urea_nitrogen,disease_stage,uncurated_metadata,family_member,subject_n,samp_n,westernized
,<fct>,<chr>,<chr>,<chr>,<chr>,<fct>,<chr>,<int>,<chr>,<chr>,⋯,<dbl>,<dbl>,<chr>,<dbl>,<int>,<chr>,<chr>,<int>,<int>,<lgl>
A11_01_1FE,PasolliE_2018,A11_01_1FE,A11_01_1FE,stool,no,control,healthy,52,adult,male,⋯,NA,NA,NA,NA,NA,NA,NA,1,1,FALSE
A13_01_1FE,PasolliE_2018,A13_01_1FE,A13_01_1FE,stool,no,control,healthy,52,adult,male,⋯,NA,NA,NA,NA,NA,NA,NA,1,1,FALSE
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
YSZC12003_37879,XieH_2016,YSZC12003_37879,YSZC12003_37879,stool,NA,control,NA,42,adult,female,⋯,NA,NA,NA,NA,NA,NA,NA,1,1,TRUE
YSZC12003_37880,XieH_2016,YSZC12003_37880,YSZC12003_37880,stool,NA,control,NA,42,adult,female,⋯,NA,NA,NA,NA,NA,NA,NA,1,1,TRUE


In [65]:
# distance-based RDA
dbRDA = capscale(d ~ westernized + gender, data = M)
dbRDA

Call: capscale(formula = d ~ westernized + gender, data = M)

                 Inertia Proportion Rank
Total         2376.96316    1.00000     
Constrained    118.51159    0.04986    2
Unconstrained 2946.75425    1.23971  651
Imaginary     -688.30269   -0.28957 1502
Inertia is squared Unknown distance 

Eigenvalues for constrained axes:
  CAP1   CAP2 
107.69  10.83 

Eigenvalues for unconstrained axes:
  MDS1   MDS2   MDS3   MDS4   MDS5   MDS6   MDS7   MDS8 
1207.3  189.6  149.9  135.0  103.7   75.8   55.8   52.2 
(Showing 8 of 651 unconstrained eigenvalues)


In [66]:
# anova
anova.cca(dbRDA, by='terms')

,Df,SumOfSqs,F,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>
westernized,1,107.02265,78.121793,0.001
gender,1,11.48894,8.386418,0.001
Residual,2151,2946.75425,NA,NA


### wunifrac-trt

In [67]:
# filtering & ordering
## distance matrix
d = filter_dist(D[['wunifrac_trt']], rownames(m))
d %>% labels %>% length
## metadata
M = m[labels(d),]
M

[1] 2154

,dataset_name,Sample,subjectID,body_site,antibiotics_current_use,study_condition,disease,age,age_category,gender,⋯,il_1,glp_1,travel_destination,urea_nitrogen,disease_stage,uncurated_metadata,family_member,subject_n,samp_n,westernized
,<fct>,<chr>,<chr>,<chr>,<chr>,<fct>,<chr>,<int>,<chr>,<chr>,⋯,<dbl>,<dbl>,<chr>,<dbl>,<int>,<chr>,<chr>,<int>,<int>,<lgl>
A11_01_1FE,PasolliE_2018,A11_01_1FE,A11_01_1FE,stool,no,control,healthy,52,adult,male,⋯,NA,NA,NA,NA,NA,NA,NA,1,1,FALSE
A13_01_1FE,PasolliE_2018,A13_01_1FE,A13_01_1FE,stool,no,control,healthy,52,adult,male,⋯,NA,NA,NA,NA,NA,NA,NA,1,1,FALSE
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
YSZC12003_37879,XieH_2016,YSZC12003_37879,YSZC12003_37879,stool,NA,control,NA,42,adult,female,⋯,NA,NA,NA,NA,NA,NA,NA,1,1,TRUE
YSZC12003_37880,XieH_2016,YSZC12003_37880,YSZC12003_37880,stool,NA,control,NA,42,adult,female,⋯,NA,NA,NA,NA,NA,NA,NA,1,1,TRUE


In [68]:
# distance-based RDA
dbRDA = capscale(d ~ westernized + gender, data = M)
dbRDA

Call: capscale(formula = d ~ westernized + gender, data = M)

                Inertia Proportion Rank
Total         204.03254    1.00000     
Constrained    14.30080    0.07009    2
Unconstrained 243.30350    1.19247  772
Imaginary     -53.57176   -0.26256 1381
Inertia is squared Unknown distance 

Eigenvalues for constrained axes:
  CAP1   CAP2 
13.304  0.997 

Eigenvalues for unconstrained axes:
 MDS1  MDS2  MDS3  MDS4  MDS5  MDS6  MDS7  MDS8 
45.74 16.24 14.11 12.86  7.69  4.96  4.52  4.22 
(Showing 8 of 772 unconstrained eigenvalues)


In [69]:
# anova
anova.cca(dbRDA, by='terms')

,Df,SumOfSqs,F,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>
westernized,1,13.178875,116.511931,0.001
gender,1,1.121923,9.918706,0.001
Residual,2151,243.303500,NA,NA


## Westernization & gender + age + BMI

In [101]:
# formatting metadata
m = meta %>%
    dt_filter(!is.na(westernized), !is.na(gender), !is.na(age), !is.na(BMI)) %>%
    as.data.frame
rownames(m) = m$Sample
m

,dataset_name,Sample,subjectID,body_site,antibiotics_current_use,study_condition,disease,age,age_category,gender,⋯,il_1,glp_1,travel_destination,urea_nitrogen,disease_stage,uncurated_metadata,family_member,subject_n,samp_n,westernized
,<fct>,<chr>,<chr>,<chr>,<chr>,<fct>,<chr>,<int>,<chr>,<chr>,⋯,<dbl>,<dbl>,<chr>,<dbl>,<int>,<chr>,<chr>,<int>,<int>,<lgl>
SID713A004.11.0.0,CosteaPI_2017,SID713A004.11.0.0,KAZ2,stool,NA,control,healthy,48,adult,female,⋯,NA,NA,NA,NA,NA,NA,NA,1,1,FALSE
SID713A009.11.90.0,CosteaPI_2017,SID713A009.11.90.0,KAZ4,stool,NA,control,healthy,46,adult,male,⋯,NA,NA,NA,NA,NA,NA,NA,2,1,FALSE
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
CCIS82944710ST.20.0,ZellerG_2014,CCIS82944710ST.20.0,FR-730,stool,NA,control,healthy,38,adult,female,⋯,NA,NA,NA,NA,NA,NA,NA,1,1,TRUE
CCIS93040568ST.20.0,ZellerG_2014,CCIS93040568ST.20.0,FR-682,stool,NA,control,healthy,65,adult,male,⋯,NA,NA,NA,NA,NA,NA,NA,1,1,TRUE


### Bray-Curtis

In [102]:
# filtering & ordering
## distance matrix
d = filter_dist(D[['bray']], rownames(m))
d %>% labels %>% length
## metadata
M = m[labels(d),]
M

[1] 935

,dataset_name,Sample,subjectID,body_site,antibiotics_current_use,study_condition,disease,age,age_category,gender,⋯,il_1,glp_1,travel_destination,urea_nitrogen,disease_stage,uncurated_metadata,family_member,subject_n,samp_n,westernized
,<fct>,<chr>,<chr>,<chr>,<chr>,<fct>,<chr>,<int>,<chr>,<chr>,⋯,<dbl>,<dbl>,<chr>,<dbl>,<int>,<chr>,<chr>,<int>,<int>,<lgl>
A11_01_1FE,PasolliE_2018,A11_01_1FE,A11_01_1FE,stool,no,control,healthy,52,adult,male,⋯,NA,NA,NA,NA,NA,NA,NA,1,1,FALSE
A13_01_1FE,PasolliE_2018,A13_01_1FE,A13_01_1FE,stool,no,control,healthy,52,adult,male,⋯,NA,NA,NA,NA,NA,NA,NA,1,1,FALSE
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
YEZ_N.8,YeZ_2018,YEZ_N.8,YEZ_N-8,stool,no,control,healthy,32,adult,male,⋯,NA,NA,NA,NA,NA,NA,NA,1,1,TRUE
YEZ_N.9,YeZ_2018,YEZ_N.9,YEZ_N-9,stool,no,control,healthy,48,adult,male,⋯,NA,NA,NA,NA,NA,NA,NA,1,1,TRUE


In [105]:
# distance-based RDA
dbRDA = capscale(d ~ westernized + gender + age + BMI, data = M)
RsquareAdj(dbRDA)

$r.squared
[1] 0.07407223

$adj.r.squared
[1] 0.07008974

In [114]:
# distance-based RDA
## effect of westernized
dbRDA = capscale(d ~ age + gender + BMI, data = M)
RsquareAdj(dbRDA)$adj.r.squared
## effect of gender
dbRDA = capscale(d ~ westernized + age + BMI, data = M)
RsquareAdj(dbRDA)$adj.r.squared
## effect of age
dbRDA = capscale(d ~ westernized + gender + BMI, data = M)
RsquareAdj(dbRDA)$adj.r.squared
## effect of BMI
dbRDA = capscale(d ~ westernized + gender + age, data = M)
RsquareAdj(dbRDA)$adj.r.squared
## all
dbRDA = capscale(d ~ westernized + gender + age + BMI, data = M)
RsquareAdj(dbRDA)$adj.r.squared

[1] 0.02151553

[1] 0.06624136

[1] 0.05522836

[1] 0.0676686

[1] 0.07008974

In [115]:
# anova
df.dims(10)
anova.cca(dbRDA, by='terms')
df.dims()

,Df,SumOfSqs,F,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>
westernized,1,13.914039,42.071836,0.001
gender,1,1.449493,4.382827,0.001
age,1,4.265807,12.898508,0.001
BMI,1,0.946974,2.863362,0.001
Residual,930,307.570518,NA,NA


### wunifrac-phy

In [116]:
# filtering & ordering
## distance matrix
d = filter_dist(D[['wunifrac_phy']], rownames(m))
d %>% labels %>% length
## metadata
M = m[labels(d),]
M

[1] 935

,dataset_name,Sample,subjectID,body_site,antibiotics_current_use,study_condition,disease,age,age_category,gender,⋯,il_1,glp_1,travel_destination,urea_nitrogen,disease_stage,uncurated_metadata,family_member,subject_n,samp_n,westernized
,<fct>,<chr>,<chr>,<chr>,<chr>,<fct>,<chr>,<int>,<chr>,<chr>,⋯,<dbl>,<dbl>,<chr>,<dbl>,<int>,<chr>,<chr>,<int>,<int>,<lgl>
A11_01_1FE,PasolliE_2018,A11_01_1FE,A11_01_1FE,stool,no,control,healthy,52,adult,male,⋯,NA,NA,NA,NA,NA,NA,NA,1,1,FALSE
A13_01_1FE,PasolliE_2018,A13_01_1FE,A13_01_1FE,stool,no,control,healthy,52,adult,male,⋯,NA,NA,NA,NA,NA,NA,NA,1,1,FALSE
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
YEZ_N.8,YeZ_2018,YEZ_N.8,YEZ_N-8,stool,no,control,healthy,32,adult,male,⋯,NA,NA,NA,NA,NA,NA,NA,1,1,TRUE
YEZ_N.9,YeZ_2018,YEZ_N.9,YEZ_N-9,stool,no,control,healthy,48,adult,male,⋯,NA,NA,NA,NA,NA,NA,NA,1,1,TRUE


In [118]:
# distance-based RDA
## effect of westernized
dbRDA = capscale(d ~ age + gender + BMI, data = M)
RsquareAdj(dbRDA)$adj.r.squared
## effect of gender
dbRDA = capscale(d ~ westernized + age + BMI, data = M)
RsquareAdj(dbRDA)$adj.r.squared
## effect of age
dbRDA = capscale(d ~ westernized + gender + BMI, data = M)
RsquareAdj(dbRDA)$adj.r.squared
## effect of BMI
dbRDA = capscale(d ~ westernized + gender + age, data = M)
RsquareAdj(dbRDA)$adj.r.squared
## all
dbRDA = capscale(d ~ westernized + gender + age + BMI, data = M)
RsquareAdj(dbRDA)$adj.r.squared

[1] 0.01688481

[1] 0.06272322

[1] 0.05659177

[1] 0.0631178

[1] 0.0663729

In [119]:
# anova
df.dims(10)
anova.cca(dbRDA, by='terms')
df.dims()

,Df,SumOfSqs,F,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>
westernized,1,53.657226,41.452484,0.001
gender,1,5.083641,3.927328,0.003
age,1,9.844317,7.605152,0.001
BMI,1,4.402015,3.400743,0.008
Residual,930,1203.817384,NA,NA


### wunifrac-trt

In [120]:
# filtering & ordering
## distance matrix
d = filter_dist(D[['wunifrac_trt']], rownames(m))
d %>% labels %>% length
## metadata
M = m[labels(d),]
M

[1] 935

,dataset_name,Sample,subjectID,body_site,antibiotics_current_use,study_condition,disease,age,age_category,gender,⋯,il_1,glp_1,travel_destination,urea_nitrogen,disease_stage,uncurated_metadata,family_member,subject_n,samp_n,westernized
,<fct>,<chr>,<chr>,<chr>,<chr>,<fct>,<chr>,<int>,<chr>,<chr>,⋯,<dbl>,<dbl>,<chr>,<dbl>,<int>,<chr>,<chr>,<int>,<int>,<lgl>
A11_01_1FE,PasolliE_2018,A11_01_1FE,A11_01_1FE,stool,no,control,healthy,52,adult,male,⋯,NA,NA,NA,NA,NA,NA,NA,1,1,FALSE
A13_01_1FE,PasolliE_2018,A13_01_1FE,A13_01_1FE,stool,no,control,healthy,52,adult,male,⋯,NA,NA,NA,NA,NA,NA,NA,1,1,FALSE
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
YEZ_N.8,YeZ_2018,YEZ_N.8,YEZ_N-8,stool,no,control,healthy,32,adult,male,⋯,NA,NA,NA,NA,NA,NA,NA,1,1,TRUE
YEZ_N.9,YeZ_2018,YEZ_N.9,YEZ_N-9,stool,no,control,healthy,48,adult,male,⋯,NA,NA,NA,NA,NA,NA,NA,1,1,TRUE


In [121]:
# distance-based RDA
## effect of westernized
dbRDA = capscale(d ~ age + gender + BMI, data = M)
RsquareAdj(dbRDA)$adj.r.squared
## effect of gender
dbRDA = capscale(d ~ westernized + age + BMI, data = M)
RsquareAdj(dbRDA)$adj.r.squared
## effect of age
dbRDA = capscale(d ~ westernized + gender + BMI, data = M)
RsquareAdj(dbRDA)$adj.r.squared
## effect of BMI
dbRDA = capscale(d ~ westernized + gender + age, data = M)
RsquareAdj(dbRDA)$adj.r.squared
## all
dbRDA = capscale(d ~ westernized + gender + age + BMI, data = M)
RsquareAdj(dbRDA)$adj.r.squared

[1] 0.02159801

[1] 0.07745127

[1] 0.06720128

[1] 0.07862464

[1] 0.08151014

In [122]:
# anova
df.dims(10)
anova.cca(dbRDA, by='terms')
df.dims()

,Df,SumOfSqs,F,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>
westernized,1,5.4638639,52.499592,0.001
gender,1,0.4770190,4.583442,0.001
age,1,1.2568415,12.076374,0.001
BMI,1,0.3405144,3.271836,0.001
Residual,930,96.7891982,NA,NA


# sessionInfo

In [91]:
sessionInfo()

R version 3.6.2 (2019-12-12)
Platform: x86_64-conda_cos6-linux-gnu (64-bit)
Running under: Ubuntu 18.04.5 LTS

Matrix products: default
BLAS/LAPACK: /ebio/abt3_projects/Georg_animal_feces/envs/phyloseq-phy/lib/libopenblasp-r0.3.7.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] LeyLabRMisc_0.1.6 vegan_2.5-6       lattice_0.20-38   permute_0.9-5    
 [5] phyloseq_1.30.0   ape_5.3           tidytable_0.4.1   data.table_1.12.8
 [9] ggplot2_3.2.1     tidyr_1.0.0       dplyr_0.8.3      

loaded via a namespace (and not attached):
 [1] Biobase_2.46.0   